<a href="https://colab.research.google.com/github/ArshanBhanage/Autogluon/blob/main/USA_Rainfall__Prediction_AutoGluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
 !pip install kagglehub[pandas-datasets]

In [7]:
slug = "waqi786/usa-rainfall-prediction-dataset-2024-2025"
dataset_dir = kagglehub.dataset_download(slug)
print(f"Dataset downloaded to:\n{dataset_dir}\n")


100%|██████████| 3.90M/3.90M [00:00<00:00, 104MB/s]

Extracting files...
Dataset downloaded to:
/root/.cache/kagglehub/datasets/waqi786/usa-rainfall-prediction-dataset-2024-2025/versions/1



In [8]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd

file_path = '/root/.cache/kagglehub/datasets/waqi786/usa-rainfall-prediction-dataset-2024-2025/versions/1/usa_rain_prediction_dataset_2024_2025.csv'
df = pd.read_csv(file_path)

# Display the first few rows to check the dataset
df.head()

,Date,Location,Temperature,Humidity,Wind Speed,Precipitation,Cloud Cover,Pressure,Rain Tomorrow
0,2024-01-01,New York,87.524795,75.655455,28.379506,0.000000,69.617966,1026.030278,0
1,2024-01-02,New York,83.259325,28.712617,12.436433,0.526995,41.606048,995.962065,0
2,2024-01-03,New York,80.943050,64.740043,14.184831,0.916884,77.364763,980.796739,1
3,2024-01-04,New York,78.097552,59.738984,19.444029,0.094134,52.541196,979.012163,0
4,2024-01-05,New York,37.059963,34.766784,3.689661,1.361272,85.584000,1031.790859,0


In [11]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━

In [12]:
import numpy as np
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

# Use the rain prediction dataset
LABEL_COL = "Rain Tomorrow"

# Split the dataset into train and test sets (80-20 split)
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

print("Train shape:", train_data.shape, " Test shape:", test_data.shape)
print("\nDataset info:")
print(df.info())
print("\nTarget distribution in training data:")
print(train_data[LABEL_COL].value_counts())

# Handle datetime column if needed
if "Date" in train_data.columns:
    train_data["Date"] = pd.to_datetime(train_data["Date"], errors="coerce")
    test_data["Date"] = pd.to_datetime(test_data["Date"], errors="coerce")

# Train AutoGluon predictor for binary classification
predictor = TabularPredictor(
    label=LABEL_COL,
    problem_type="binary",  # Rain Tomorrow is binary (0 or 1)
    eval_metric="roc_auc"
).fit(
    train_data=train_data,
    presets="best_quality",
    time_limit=3600,
)

# Display leaderboard
lb = predictor.leaderboard(silent=True)
display(lb.head(10))

# Make predictions on test set
preds = predictor.predict(test_data.drop(columns=[LABEL_COL]))
pred_probs = predictor.predict_proba(test_data.drop(columns=[LABEL_COL]))

# Evaluate performance
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_true = test_data[LABEL_COL]
accuracy = accuracy_score(y_true, preds)
roc_auc = roc_auc_score(y_true, pred_probs[1])

print(f"\n=== Test Set Performance ===")
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_true, preds))

# Show sample predictions
results = test_data.copy()
results['Predicted_Rain'] = preds
results['Probability_Rain'] = pred_probs[1]
display(results[[LABEL_COL, 'Predicted_Rain', 'Probability_Rain', 'Temperature', 'Humidity', 'Pressure']].head(20))


No path specified. Models will be saved in: "AutogluonModels/ag-20251102_004835"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Memory Avail:       11.28 GB / 12.67 GB (89.0%)
Disk Space Avail:   62.16 GB / 107.72 GB (57.7%)
Presets specified: ['best_quality']


Train shape: (58480, 9)  Test shape: (14620, 9)

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73100 entries, 0 to 73099
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           73100 non-null  object 
 1   Location       73100 non-null  object 
 2   Temperature    73100 non-null  float64
 3   Humidity       73100 non-null  float64
 4   Wind Speed     73100 non-null  float64
 5   Precipitation  73100 non-null  float64
 6   Cloud Cover    73100 non-null  float64
 7   Pressure       73100 non-null  float64
 8   Rain Tomorrow  73100 non-null  int64  
dtypes: float64(6), int64(1), object(2)
memory usage: 5.0+ MB
None

Target distribution in training data:
Rain Tomorrow
0    45619
1    12861
Name: count, dtype: int64


Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.
	Running DyStack for up to 900s of the 3600s of remaining time (25%).
	Running DyStack sub-fit in a ray process to avoid memory leakage. Enabling ray logging (enable_ray_logging=True). Specify `ds_args={'enable_ray_logging': False}` if you experience logging issues.
2025-11-02 00:48:46,764	INFO worker.py:1843 -- Started a local Ray instance. V

(_ray_fit pid=4981) [1000]	valid_set's binary_logloss: 0.0069538
(_ray_fit pid=4982) [2000]	valid_set's binary_logloss: 0.00492731 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=4982) [3000]	valid_set's binary_logloss: 0.00413845 [repeated 2x across cluster]
(_ray_fit pid=5216) [1000]	valid_set's binary_logloss: 0.00836644
(_ray_fit pid=4982) [4000]	valid_set's binary_logloss: 0.0037367
(_ray_fit pid=4982) [5000]	valid_set's binary_logloss: 0.00289454
(_ray_fit pid=5216) [2000]	valid_set's binary_logloss: 0.00735772
(_ray_fit pid=4982) [6000]	valid_set's binary_logloss: 0.00243531
(_ray_fit pid=5216) [3000]	valid_set's binary_logloss: 0.0075436
(_ray_fit pid=4982) [7000]	valid_set's binary_logloss: 0.00204967
(_ray_fit pid=4982) [8000]	valid_set's binary_logloss: 0.00180575
(_ray_

(_dystack pid=4751) 	1.0	 = Validation score   (roc_auc)
(_dystack pid=4751) 	234.17s	 = Training   runtime
(_dystack pid=4751) 	43.34s	 = Validation runtime
(_dystack pid=4751) Fitting model: LightGBM_BAG_L1 ... Training model for up to 342.96s of the 637.78s of remaining time.
(_dystack pid=4751) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.35%)
(_dystack pid=4751) 	1.0	 = Validation score   (roc_auc)
(_dystack pid=4751) 	49.37s	 = Training   runtime
(_dystack pid=4751) 	0.66s	 = Validation runtime
(_dystack pid=4751) Fitting model: RandomForestGini_BAG_L1 ... Training model for up to 287.34s of the 582.16s of remaining time.
(_dystack pid=4751) 	1.0	 = Validation score   (roc_auc)
(_dystack pid=4751) 	22.99s	 = Training   runtime
(_dystack pid=4751) 	1.4s	 = Validation runtime
(_dystack pid=4751) Fitting model: RandomForestEntr_BAG_L1 ... Training model for up to 262.74s of the 557.57s of remaining ti

...

(_ray_fit pid=9750) [1000]	valid_set's binary_logloss: 3.17553e-07
(_ray_fit pid=9750) [2000]	valid_set's binary_logloss: 3.17072e-07
(_ray_fit pid=9750) [3000]	valid_set's binary_logloss: 3.16798e-07
(_ray_fit pid=9750) [4000]	valid_set's binary_logloss: 3.16727e-07
(_ray_fit pid=9750) [5000]	valid_set's binary_logloss: 3.16697e-07
(_ray_fit pid=9750) [6000]	valid_set's binary_logloss: 3.16705e-07


(_ray_fit pid=9750) 	Ran out of time, early stopping on iteration 6989. Best iteration is:
(_ray_fit pid=9750) 	[6984]	valid_set's binary_logloss: 3.16679e-07
(_dystack pid=4751) 	1.0	 = Validation score   (roc_auc)
(_dystack pid=4751) 	92.95s	 = Training   runtime
(_dystack pid=4751) 	3.28s	 = Validation runtime
(_dystack pid=4751) Fitting model: RandomForestGini_BAG_L2 ... Training model for up to 85.13s of the 85.07s of remaining time.
(_dystack pid=4751) 	1.0	 = Validation score   (roc_auc)
(_dystack pid=4751) 	14.0s	 = Training   runtime
(_dystack pid=4751) 	1.69s	 = Validation runtime
(_dystack pid=4751) Fitting model: RandomForestEntr_BAG_L2 ... Training model for up to 69.25s of the 69.19s of remaining time.
(_dystack pid=4751) 	1.0	 = Validation score   (roc_auc)
(_dystack pid=4751) 	13.21s	 = Training   runtime
(_dystack pid=4751) 	1.62s	 = Validation runtime
(_dystack pid=4751) Fitting model: CatBoost_BAG_L2 ... Training model for up to 54.22s of the 54.17s of remaining time

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr_BAG_L1,1.0,roc_auc,1.437270,26.951090,1.437270,26.951090,1,True,4
1,RandomForestGini_BAG_L1,1.0,roc_auc,1.447120,21.702033,1.447120,21.702033,1,True,3
2,WeightedEnsemble_L2,1.0,roc_auc,3.694111,38.878718,0.008831,3.553749,2,True,12
3,WeightedEnsemble_L3,1.0,roc_auc,3.696744,41.818382,0.011464,6.493413,3,True,22
4,ExtraTreesGini_BAG_L2,1.0,roc_auc,52.983920,1707.892410,1.511237,4.342259,2,True,18
5,ExtraTreesEntr_BAG_L2,1.0,roc_auc,53.042386,1709.361030,1.569702,5.810879,2,True,19
6,NeuralNetFastAI_BAG_L2,1.0,roc_auc,54.355708,1998.071872,2.883025,294.521721,2,True,20
7,RandomForestGini_BAG_L2,1.0,roc_auc,53.012824,1719.211333,1.540141,15.661182,2,True,15
8,RandomForestEntr_BAG_L2,1.0,roc_auc,52.960029,1718.719462,1.487346,15.169311,2,True,16
9,XGBoost_BAG_L2,1.0,roc_auc,52.089748,1750.345860,0.617065,46.795709,2,True,21



=== Test Set Performance ===
Accuracy: 1.0000
ROC AUC: 1.0000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11369
           1       1.00      1.00      1.00      3251

    accuracy                           1.00     14620
   macro avg       1.00      1.00      1.00     14620
weighted avg       1.00      1.00      1.00     14620



,Rain Tomorrow,Predicted_Rain,Probability_Rain,Temperature,Humidity,Pressure
66127,0,0,0.011111,48.052607,60.603176,1020.133430
43485,0,0,0.022222,76.409199,87.441871,1018.283834
34304,1,1,0.931111,35.668016,68.053619,1020.116697
34317,0,0,0.103333,63.633611,83.717856,1001.981662
38631,0,0,0.030000,51.459740,82.835612,976.618916
42111,0,0,0.021111,81.816706,88.999969,982.168164
63601,1,1,0.821111,47.541151,61.754876,971.909811
30662,0,0,0.035556,37.718925,43.696166,1016.566241
21439,0,0,0.016667,37.666887,82.118263,996.485480
12107,0,0,0.025556,93.376145,61.142612,997.736175
